#### Estimating BIP parameters [link](https://cheguide.com/estimate_BIP.html)

In [ ]:
# all parameters we need
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import minimize as mi
R = 8.31446261815324

In [ ]:
p = 0.10133 #Mpa
mw = np.array([58.08, 119.378])
pc = np.array([4.70148, 5.47155]) #Mpa
tc = np.array([508.1, 536.4])
w = np.array([0.309, 0.216])

#txy data
t = np.array([335.75, 336.65, 337.25, 337.55, 336.95, 335.85, 334.65, 333.45, 332.15, 331.05, 330.15])
# t = np.array([335.75])
x1 = np.array([0.098, 0.186, 0.266, 0.360, 0.468, 0.578, 0.673, 0.755, 0.827, 0.892, 0.949])
# x1 = np.array([0.098])
x2 = 1 - x1
y1 = np.array([0.060, 0.143, 0.230, 0.360, 0.514, 0.646, 0.751, 0.830, 0.890, 0.939, 0.975])
y2 = 1 - y1

ki = 0.37464 + 1.54226*w - 0.26992*w**2
k_exp = y1 / x1

In [ ]:
plt.xlabel("x [liq phase]")
plt.ylabel("y [gas phase]")
plt.plot(x1,y1,'r--',label='Acetone')
plt.plot(x2,y2,'b:',label="chloroform")
plt.title("T-x-y Plot")
plt.legend()
plt.show()

In [ ]:
# k12 = -0.0604948725819791
k12 = 0.12
x1.size
kk = np.zeros((2,2))
kk[0][0] = 0
kk[0][1] = k12
kk[1][0] = k12
kk[1][1] = 0


In [ ]:
alphai = np.zeros((2,x1.size))
ai = np.zeros((2,x1.size))
bi = np.zeros((2,x1.size))

for i in range(2):
    alphai[i] = (1 + ki[i] * (1 - (t/tc[i])**0.5))**2
    ai[i] = 0.45724 * (R * tc[i])**2 * alphai[i] / pc[i]
    bi[i] = 0.07780 * R * tc[i] / pc[i]

In [ ]:
aik = np.zeros((2,x1.size))
a12 = ((ai[0] * ai[1])**0.5 * (1 - k12))
aik[0] = (ai[0] * x1) + (a12 * x2)
aik[1] = (ai[1] * x2) + (a12 * x1)

In [ ]:
bi

In [ ]:
for i in range(0,2):
    for j in range(0, 2):
        print(i,j,((ai[i] * ai[j])**0.5 * (1 - kk[i][j])))

In [ ]:
# for liq phase
a = ai[0]*x1**2 + (2 * a12 * x1 * x2) + ai[1]*x2**2
b = bi[0]*x1 + bi[1]*x2
A = a*p/(R*t)**2
B = (b*p)/(R*t)
C2 = (B-1)
C1 = (A - 3*B**2 - 2*B)
C0 = (B**3 + B**2 - A*B)

In [ ]:
aik[0]

In [ ]:
# for liq phase
phiL = []
zl = np.zeros(x1.size)
l=0
for i,j,k in zip(C2,C1,C0):
    coeff = np.array([1,i,j,k])
    roots = np.array(np.roots(coeff))
    zl[l] = roots[2]
    l+=1

# print(zl)
temp1 = bi / b
phiL = np.exp(((temp1) * (zl - 1)) -  (np.log(zl - B)) - (A / (2*np.sqrt(2)*B)) *  ((2 * aik / a) - temp1) * np.log((zl + 2.414213562373095*B) / (zl - 0.41421356237309515*B)))
    

print(phiL)


In [ ]:
aik[0]

In [ ]:
# for gas phase
a = ai[0]*y1**2 + (2 * a12 * y1 * y2) + ai[1]*y2**2
b = bi[0]*y1 + bi[1]*y2
A = a*p/(R*t)**2
B = (b*p)/(R*t)
C2 = (B-1)
C1 = (A - 3*B**2 - 2*B)
C0 = (B**3 + B**2 - A*B)

In [ ]:
# for gas phase
phiV = []
for i,j,k in zip(C2,C1,C0):
    coeff = np.array([1,i,j,k])
    roots = np.array(np.roots(coeff))
    zv = roots[0]
    # print(zv)
    temp1 = b / bi
    phiV = np.exp(((temp1) * (zl - 1)) -  (np.log(zl - B)) - (0.35355339059327373 * A / B) *  ((2 * aik / a) - temp1) * np.log((zl + 2.414213562373095*B) / (zl - 0.41421356237309515*B)))
print(phiV)



In [ ]:
# common parameters
alphai = np.zeros((2,x1.size))
ai = np.zeros((2,x1.size))
bi = np.zeros((2,x1.size))

for i in range(2):
    alphai[i] = (1 + ki[i] * (1 - (t/tc[i])**0.5))**2
    ai[i] = 0.45724 * (R * tc[i])**2 * alphai[i] / pc[i]
    bi[i] = 0.07780 * R * tc[i] / pc[i]

# def func(k12, to_graph=False):
def func(k12):
    aik = np.zeros((2,x1.size))
    a12 = ((ai[0] * ai[1])**0.5 * (1 - k12))
    aik[0] = (ai[0] * x1) + (a12 * x2)
    aik[1] = (ai[1] * x2) + (a12 * x1)

    # PR props for liq phase
    a = ai[0]*x1**2 + (2 * a12 * x1 * x2) + ai[1]*x2**2
    b = bi[0]*x1 + bi[1]*x2
    A = a*p/(R*t)**2
    B = (b*p)/(R*t)
    C2 = (B-1)
    C1 = (A - 3*B**2 - 2*B)
    C0 = (B**3 + B**2 - A*B)

    # find fugacity liq phase
    phiL = []
    zl = np.zeros(x1.size)
    l=0
    for i,j,k in zip(C2,C1,C0):
        coeff = np.array([1,i,j,k])
        roots = np.array(np.roots(coeff))
        zl[l] = roots[2]
        l+=1
    # print(zl)
    temp1 = bi / b
    phiL = np.exp(((temp1) * (zl - 1)) -  (np.log(zl - B)) - (A / (2*np.sqrt(2)*B)) *  ((2 * aik / a) - temp1) * np.log((zl + 2.414213562373095*B) / (zl - 0.41421356237309515*B)))

    aik[0] = (ai[0] * y1) + (a12 * y2)
    aik[1] = (ai[1] * y2) + (a12 * y1)
    
    # PR props for gas phase
    a = ai[0]*y1**2 + (2 * a12 * y1 * y2) + ai[1]*y2**2
    b = bi[0]*y1 + bi[1]*y2
    A = a*p/(R*t)**2
    B = (b*p)/(R*t)
    C2 = (B-1)
    C1 = (A - 3*B**2 - 2*B)
    C0 = (B**3 + B**2 - A*B)

    # find fugacity gas phase
    phiV = []
    zv = np.zeros(y1.size)
    l=0
    for i,j,k in zip(C2,C1,C0):
        coeff = np.array([1,i,j,k])
        roots = np.array(np.roots(coeff))
        zv[l] = roots[0]
        l+=1
    # print(zl)
    temp1 = bi / b
    phiV = np.exp(((temp1) * (zv - 1)) -  (np.log(zv - B)) - (A / (2*np.sqrt(2)*B)) *  ((2 * aik / a) - temp1) * np.log((zv + 2.414213562373095*B) / (zv - 0.41421356237309515*B)))

    k_calc = phiL/phiV
    y1_calc = k_calc[0]*x1
    # print(k_calc[0] - k_exp)
    # return (np.linalg.norm(y1-y1_calc)**2)/len(y1)
    return (np.abs(k_exp-k_calc[0])).sum()
    # print(phiL)
    # print(phiV)

    # if(to_graph):
        # plt.plot(x1,y1_calc,label="predicted")
        # plt.plot(x1,y1,'r:',label="original")
        # plt.legend()


In [ ]:
func(0.12)

In [ ]:
x0 = 0.12
res = mi(func, x0, method='Nelder-Mead', tol=1e-5)

In [ ]:
res